In [57]:

def create_equity_curve(df,z0,z1,pos):
    ec=df.diff(axis=0)
    ec.iloc[0]=0
    
    i=1
    while i<len(ec):
        ec.iloc[i]=ec.iloc[i-1]+ec.iloc[i]*z0*z1*pos
        i+=1
    return ec
    
    

def pos(a):
    pos=1
    
    if a=='y':
        pos=1
        
    if a=='n':
        pos=pos*(-1)
        
    else:
        print "Please rerun the code and input either 'y' or 'n' for position"
        
    return pos

def z_factor(product1,product2):
    z=1
    factor=1000
    if product1=='RB' and product2 =='RB':
        z=42
    elif product1=='HO' and product2 =='HO':
        z=42
    elif product1=='RB' and product2 =='HO':
        z=42
    elif product1=='HO' and product2 =='RB':
        z=42
    elif product1=='G' and product2 =='G':
        factor=100
    elif product1=='G' and product2 =='B':
        z=(1/7.45)
    
    else:
        z=1
        factor=1000
    
    return z,factor
        
def create_sprd(dataframe1, dataframe2,sprd_type):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
        
    if sprd_type==1:
            
        new_df=dataframe1*42-dataframe2
        new_df['Date']=date
        new_df.set_index('Date',inplace=True)
        new_df.fillna(method='bfill',inplace=True)
        new_df.fillna(method='ffill',inplace=True)
            
        return new_df
        
        
    elif sprd_type==2:
            
        new_df= dataframe1-dataframe2
        new_df['Date']=date
        new_df.set_index('Date',inplace=True)
        new_df.fillna(method='bfill',inplace=True)
        new_df.fillna(method='ffill',inplace=True)
            
        return new_df

        
    else:
        print 'You have inputted incorrectly. Please rerun and enter either integer 1 or 2 for spread type.'
    
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i,j,loop_start,loop_end):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            
            start_date_new=dates[0]+loop_start
            end_date_new=dates[-1]+loop_end
            
            new_dates=pd.date_range(start_date_new,end_date_new)
            
            new_dates=new_dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(new_dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':new_dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date_new] # filter for start date

            temp=temp[temp['Date']<=end_date_new] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi,seasonal_year,start_date_new
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i,j,loop_start,loop_end):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
 
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
        
            start_date_new=dates[0]+loop_start
            end_date_new=dates[-1]+loop_end
            
            new_dates=pd.date_range(start_date_new,end_date_new)

            new_dates=new_dates.strftime('%Y-%m-%d') #make sure no left out dates

            sample_data=np.arange(0,len(new_dates)) #just sample numbers to create dataframe
            dates_dataframe=pd.DataFrame({'Date':new_dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date_new] # filter for start date

            temp=temp[temp['Date']<=end_date_new] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi,seasonal_year,end_date_new
    
    
import numpy as np
import pandas as pd
import os


%matplotlib inline

#user specify instrument and contract month

#create two lists to store the start dates and end dates - later to rename the multi-index table
sd_list=[]
ed_list=[]

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

%cd Quandl
%cd $product1
%cd $month1

start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))

sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
pos_input=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()


print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

days_to_loop_start=int(raw_input('Please input the number of days to loop on start date: '))
days_to_loop_end=int(raw_input('Please input the number of days to loop on end date: '))

loop_start=-days_to_loop_start
loop_end=-days_to_loop_end

#creating the multi-index for pnl
outside=[a for a in range(loop_start,-loop_start+1)]
len_outside=len(outside)

inside=[a for a in range(loop_end,-loop_end+1)]
len_inside=len(inside)

inside=inside*len_outside

outside=outside*len_inside
outside.sort()


hier_index=list(zip(outside, inside))
hier_index=pd.MultiIndex.from_tuples(hier_index)

years_list=[]

path = os.getcwd()
listing = os.listdir(path)

for infile in listing: # get list of all years to build dataframe
    
    rd_file=pd.read_excel(infile) #read each file into panda dataframe
    yr_str=infile[-8:-4] # get the contract year
    years_list.append(yr_str)

pnl_loop=pd.DataFrame(np.random.randn(len(inside),len(years_list)),hier_index,years_list) 
##pnl multi index created

%cd ..
%cd ..
%cd ..

first_index=0
second_index=0

while first_index<=days_to_loop_start*2:
    second_index=0
    loop_end=-days_to_loop_end
    
    while second_index<=days_to_loop_end*2:
        %cd Quandl
        %cd $product1
        %cd $month1

        front_month,sy,sdn=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i,j,loop_start, loop_end)
        %cd ..
        %cd ..
        %cd ..

        %cd Quandl
        %cd $product2
        %cd $month2

        back_month,esd,edn=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i,j,loop_start, loop_end)
        %cd ..
        %cd ..
        %cd ..

        Spread_Combi=create_sprd(front_month, back_month,sprd_type)
        Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs
        Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

        position=pos(pos_input)

        zf=z_factor(product1, product2)

        #PnL Series
        pnl_series=(Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])*zf[0]*zf[1]*position
        pnl_loop.loc[loop_start].loc[loop_end]=pnl_series
        
        loop_end+=1
        second_index+=1
        ed_list.append(edn.strftime('%m-%d'))
        
    loop_start+=1
    first_index+=1
    sd_list.append(sdn.strftime('%m-%d'))
 
pnl_loop.index.names=['Start', 'End']
pnl_loop.index.set_levels([sd_list,ed_list],inplace=True)


Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 5
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 01
Please input 1 if start mth is the same as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 01
Please input end day of seasonal period in DD format: 10
Please input 1 if end mth is the same as contract year and 0 if not: 1
Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 1
Please inp

C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl\CL\Mar
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl\CL\Mar
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Doc

C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl\CL\Mar
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Doc

In [59]:
Win_Percent=(pnl_loop[pnl_loop>0].count(axis=1))/(pnl_loop.count(axis=1))

In [60]:
Win_Percent.sort_values(ascending=False)

Start  End  
12-30  01-08    0.608696
01-01  01-08    0.608696
01-02  01-08    0.608696
01-03  01-08    0.565217
01-01  01-09    0.565217
12-31  01-08    0.565217
01-02  01-09    0.565217
12-30  01-09    0.565217
12-29  01-08    0.565217
01-03  01-09    0.565217
01-04  01-08    0.565217
       01-09    0.565217
12-31  01-09    0.565217
12-29  01-12    0.521739
       01-10    0.521739
       01-09    0.521739
01-04  01-10    0.521739
12-31  01-10    0.478261
12-30  01-10    0.478261
       01-11    0.478261
       01-12    0.478261
01-01  01-10    0.478261
01-02  01-10    0.478261
01-03  01-10    0.478261
12-31  01-11    0.434783
       01-12    0.434783
01-01  01-11    0.434783
       01-12    0.434783
01-02  01-11    0.434783
       01-12    0.434783
12-29  01-11    0.434783
01-04  01-12    0.434783
       01-11    0.391304
01-03  01-11    0.391304
       01-12    0.391304
dtype: float64

In [61]:
pwd

u'C:\\Users\\boonsin.teo\\Documents'